# Predicting Delivery Times

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = None

## Loading the Data

The data comes from a Kaggle dataset on a Brazilian e-commerce company: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce

In [2]:
df_customers = pd.read_csv('Final_Project_Data/olist_customers_dataset.csv') 
df_geolocation = pd.read_csv('Final_Project_Data/olist_geolocation_dataset.csv') 
df_order_items = pd.read_csv('Final_Project_Data/olist_order_items_dataset.csv') 
df_order_payments = pd.read_csv('Final_Project_Data/olist_order_payments_dataset.csv') 
df_order_reviews = pd.read_csv('Final_Project_Data/olist_order_reviews_dataset.csv') 
df_orders = pd.read_csv('Final_Project_Data/olist_orders_dataset.csv', parse_dates=['order_purchase_timestamp','order_delivered_customer_date','order_estimated_delivery_date']) 
df_products = pd.read_csv('Final_Project_Data/olist_products_dataset.csv') 
df_sellers = pd.read_csv('Final_Project_Data/olist_sellers_dataset.csv') 
df_product_catname_translation = pd.read_csv('Final_Project_Data/product_category_name_translation.csv') 

### Merging the Dataframes

The data was split into several dataframes based on topic. There is a dataframe for order information, for customer information, for seller information, etc. Consequently, several dataframes had to be combined to get all the information needed for the analysis. 

In [3]:
df_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [4]:
df_orders = df_orders.merge(df_order_items, on='order_id')

In [5]:
df_orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,1,f1d4ce8c6dd66c47bbaa8c6781c2a923,1f9ab4708f3056ede07124aad39a2554,2018-02-12 13:10:37,174.90,20.10
112646,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,1,b80910977a37536adeddd63663f916ad,d50d79cb34e38265a8649c383dcffd48,2017-09-05 15:04:16,205.99,65.02
112647,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,1,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59
112648,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,2,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-01-12 21:36:21,179.99,40.59


In [6]:
df_orders = df_orders.merge(df_products, on='product_id')
product_map = df_product_catname_translation.set_index('product_category_name')['product_category_name_english']
product_map = product_map.to_dict()
df_orders.product_category_name = df_orders.product_category_name.map(product_map)

In [7]:
df_orders = df_orders.merge(df_customers, on='customer_id')


In [8]:
df_orders = df_orders.merge(df_sellers, on='seller_id')

## Feature Creation and Selection 

First, we want to limit the data to only orders that were completed, as we will not be able to train our data on orders that are cancelled or still processing. 

In [9]:
df_orders.order_status.value_counts()

delivered      110197
shipped          1185
canceled          542
invoiced          359
processing        357
unavailable         7
approved            3
Name: order_status, dtype: int64

In [10]:
delivered_items = df_orders[df_orders.order_status == 'delivered']

In [11]:
delivered_items.shape

(110197, 29)

Next, we will select the columns we will use to make the predictions. In particular, we will select columns with information about the seller location, customer location, product dimensions, and product contents. 

In [12]:
delivered_items.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,housewares,40.0,268.0,4.0,500.0,19.0,8.0,13.0,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,9350,maua,SP
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-08-21 20:05:16,29.99,7.78,housewares,40.0,268.0,4.0,500.0,19.0,8.0,13.0,3a51803cc0d012c3b5dc8b7528cb05f7,3366,sao paulo,SP,9350,maua,SP
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-08-08 18:37:31,29.99,7.78,housewares,40.0,268.0,4.0,500.0,19.0,8.0,13.0,ef0996a1a279c26e7ecbd737be23d235,2290,sao paulo,SP,9350,maua,SP
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,delivered,2017-10-23 23:26:46,2017-10-25 02:14:11,2017-10-27 16:48:46,2017-11-07 18:04:59,2017-11-13,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-31 02:14:11,29.99,14.10,housewares,40.0,268.0,4.0,500.0,19.0,8.0,13.0,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,9350,maua,SP
4,8736140c61ea584cb4250074756d8f3b,ab8844663ae049fda8baf15fc928f47f,delivered,2017-08-10 13:35:55,2017-08-10 13:50:09,2017-08-11 13:52:35,2017-08-16 19:03:36,2017-08-23,1,b00a32a0b42fd65efb58a5822009f629,3504c0cb71d7fa48d967e0e4c94d59d9,2017-08-16 13:50:09,75.90,7.79,baby,58.0,398.0,3.0,238.0,20.0,10.0,15.0,02c9e0c05a817d4562ec0e8c90f29dba,8577,itaquaquecetuba,SP,9350,maua,SP


In [13]:
delivered_items = delivered_items[['order_purchase_timestamp',
        'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'shipping_limit_date',
       'price', 'freight_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'seller_zip_code_prefix', 'seller_city', 'seller_state']]

In [14]:
delivered_items.head()

,order_purchase_timestamp,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_zip_code_prefix,customer_city,customer_state,seller_zip_code_prefix,seller_city,seller_state
0,2017-10-02 10:56:33,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017-10-06 11:07:15,29.99,8.72,housewares,40.0,268.0,4.0,500.0,19.0,8.0,13.0,3149,sao paulo,SP,9350,maua,SP
1,2017-08-15 18:29:31,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28,2017-08-21 20:05:16,29.99,7.78,housewares,40.0,268.0,4.0,500.0,19.0,8.0,13.0,3366,sao paulo,SP,9350,maua,SP
2,2017-08-02 18:24:47,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15,2017-08-08 18:37:31,29.99,7.78,housewares,40.0,268.0,4.0,500.0,19.0,8.0,13.0,2290,sao paulo,SP,9350,maua,SP
3,2017-10-23 23:26:46,2017-10-27 16:48:46,2017-11-07 18:04:59,2017-11-13,2017-10-31 02:14:11,29.99,14.10,housewares,40.0,268.0,4.0,500.0,19.0,8.0,13.0,88032,florianopolis,SC,9350,maua,SP
4,2017-08-10 13:35:55,2017-08-11 13:52:35,2017-08-16 19:03:36,2017-08-23,2017-08-16 13:50:09,75.90,7.79,baby,58.0,398.0,3.0,238.0,20.0,10.0,15.0,8577,itaquaquecetuba,SP,9350,maua,SP


In [15]:
delivered_items.customer_city.value_counts()

sao paulo               17400
rio de janeiro           7592
belo horizonte           3087
brasilia                 2341
curitiba                 1727
                        ...  
rio grande do piaui         1
desterro do melo            1
antonio goncalves           1
ribeirao cascalheira        1
canto do buriti             1
Name: customer_city, Length: 4085, dtype: int64

### Measuring distance between cities

One particularly important feature for our predictions would be distance between the customer and the seller. This information is not explicitly listed. However, we do know the city of the seller and the city of the buyer. We can find the approximate distance by getting the latitude and longitude of both cities, and then measuring the distance between them. 

In [16]:
city_locations = df_geolocation.groupby('geolocation_city')['geolocation_city','geolocation_lat','geolocation_lng'].first()
city_locations.reset_index(drop=True,inplace=True)

/tmp/ipykernel_384830/940603109.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  city_locations = df_geolocation.groupby('geolocation_city')['geolocation_city','geolocation_lat','geolocation_lng'].first()


In [17]:
delivered_items = delivered_items.merge(city_locations, left_on='customer_city', right_on='geolocation_city', how='left')
delivered_items.rename({'geolocation_lat': 'customer_lat', 'geolocation_lng': 'customer_long'},axis=1, inplace=True)
delivered_items = delivered_items.merge(city_locations, left_on='seller_city', right_on='geolocation_city', how='left')
delivered_items.rename({'geolocation_lat': 'seller_lat', 'geolocation_lng': 'seller_long'},axis=1, inplace=True)

In [18]:
import math

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # radius of the earth in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

# apply the distance calculation function to the dataframe
delivered_items['distance'] = delivered_items.apply(lambda row: calculate_distance(row['customer_lat'], row['customer_long'], row['seller_lat'], row['seller_long']), axis=1)



In [19]:
delivered_items = delivered_items[['order_purchase_timestamp',
       'order_delivered_customer_date', 'order_estimated_delivery_date', 'price', 'freight_value',
       'product_category_name', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
        'customer_city', 'customer_state',
        'seller_city', 'seller_state','distance']]

In [20]:
delivered_items.tail(10)

,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,price,freight_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_city,customer_state,seller_city,seller_state,distance
110187,2018-07-25 16:18:46,2018-07-30 19:42:42,2018-08-13,89.90,18.73,furniture_decor,450.0,16.0,16.0,16.0,araxa,MG,marilia,SP,425.698898
110188,2017-07-24 11:29:43,2017-08-10 19:42:10,2017-08-23,29.40,17.92,auto,700.0,16.0,7.0,11.0,vitoria da conquista,BA,sao paulo,SP,1140.413178
110189,2017-11-20 12:49:35,2017-12-27 00:06:56,2017-12-18,59.90,19.66,NaN,1500.0,30.0,10.0,36.0,campo verde,MT,ibitinga,SP,961.521479
110190,2017-03-15 11:41:46,2017-03-20 10:34:54,2017-04-03,45.00,10.96,books_technical,300.0,28.0,2.0,18.0,santo andre,SP,osasco,SP,24.849316
110191,2018-08-10 21:14:35,2018-08-21 04:16:31,2018-08-30,44.99,22.25,sports_leisure,600.0,30.0,20.0,20.0,carai,MG,paulo lopes,SC,1395.064940
110192,2018-08-10 21:14:35,2018-08-21 04:16:31,2018-08-30,44.99,22.25,sports_leisure,600.0,30.0,20.0,20.0,carai,MG,paulo lopes,SC,1395.064940
110193,2018-07-01 10:23:10,2018-07-09 15:06:57,2018-07-20,79.00,14.13,construction_tools_lights,750.0,30.0,28.0,28.0,ferraz de vasconcelos,SP,porto ferreira,SP,218.573444
110194,2018-07-24 09:46:27,2018-08-02 22:47:35,2018-08-16,399.00,45.07,furniture_decor,2100.0,80.0,8.0,30.0,fortaleza,CE,americana,SP,2317.965145
110195,2018-05-22 21:13:21,2018-06-12 23:11:29,2018-06-08,219.90,24.12,furniture_decor,5900.0,41.0,21.0,41.0,teofilo otoni,MG,sao paulo,SP,826.147138
110196,2018-05-15 17:41:00,2018-05-21 14:31:41,2018-05-29,15.50,12.79,perfumery,83.0,17.0,8.0,13.0,sao bernardo do campo,SP,ribeirao preto,SP,288.967171


### Additional feature selection

Below, we round the target variable (delivery date) to the nearest day. Additionally, we find the predicted delivery date that the e-commerce company had for each order. 

In [21]:
delivered_items['target'] = (delivered_items.order_delivered_customer_date - delivered_items.order_purchase_timestamp).dt.round('d').dt.days
delivered_items['olist_prediction'] = (delivered_items.order_estimated_delivery_date - delivered_items.order_purchase_timestamp).dt.round('d').dt.days
delivered_items['olist_error'] = abs(delivered_items.target - delivered_items.olist_prediction)

Beyond distance, we also believe that the actual city of the seller and customer should be included in the model. Some cities might be particularly hard to navigate, or surrounded by mountains that make it take longer to get to them, so it would be relevant to include the cities in addition to the distance.

In [22]:
len(delivered_items.customer_city.unique())

4085

In [23]:
delivered_items.customer_city.value_counts()

sao paulo               17400
rio de janeiro           7592
belo horizonte           3087
brasilia                 2341
curitiba                 1727
                        ...  
rio grande do piaui         1
desterro do melo            1
antonio goncalves           1
ribeirao cascalheira        1
canto do buriti             1
Name: customer_city, Length: 4085, dtype: int64

In [24]:
len(delivered_items.seller_city.unique())

595

In [25]:
delivered_items.seller_city.value_counts()

sao paulo                27357
ibitinga                  7621
curitiba                  2955
santo andre               2886
sao jose do rio preto     2544
                         ...  
messias targino              1
barro alto                   1
cascavael                    1
campos novos                 1
bom jesus dos perdoes        1
Name: seller_city, Length: 595, dtype: int64

However, there are over 4,000 unique cities in the dataset. It would be challenging to include all of these cities in our model, but thankfully many customers and sellers come from a handful of the most populous cities. Consequently, we will only include the top 15 customer and seller cities in the model, along with an "other" category for the remainder. 

In [26]:
def reduce__customer_cities(x):
    my_cities = ['sao paulo', 'rio de janeiro', 'belo horizonte', 'brasilia', 'curitiba',
       'campinas', 'porto alegre', 'salvador', 'guarulhos',
       'sao bernardo do campo', 'niteroi', 'santo andre', 'osasco', 'santos',
       'goiania'] #top 15 customer cities
    if x in my_cities:
        return x
    else:
        return 'other'

In [27]:
delivered_items.customer_city = delivered_items.customer_city.apply(reduce__customer_cities)

In [28]:
def reduce__seller_cities(x):
    my_cities = ['sao paulo', 'ibitinga', 'curitiba', 'santo andre',
       'sao jose do rio preto', 'belo horizonte', 'rio de janeiro', 'maringa',
       'ribeirao preto', 'guarulhos', 'piracicaba', 'itaquaquecetuba',
       'campinas', 'salto', 'praia grande'] #top 15 seller cities 
    if x in my_cities:
        return x
    else:
        return 'other'

In [29]:
delivered_items.seller_city = delivered_items.seller_city.apply(reduce__seller_cities)

Another possible feature that could be relevant is day of week and month. Month could be relevant if some months generally have worse weather or heavier traffic that could slow delivery times. Day of week could be relevant if orders are processed faster on certain days. For instance, orders made on the weekend might take longer to process than those made on weekdays. 

In [30]:
delivered_items['month'] = delivered_items.order_purchase_timestamp.dt.month

In [31]:
delivered_items['day_of_week'] = delivered_items.order_purchase_timestamp.dt.day_of_week

In [32]:
delivered_items.dropna(inplace=True)

In [33]:
delivered_items.tail(10)

,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,price,freight_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,customer_city,customer_state,seller_city,seller_state,distance,target,olist_prediction,olist_error,month,day_of_week
110186,2018-02-25 17:37:45,2018-03-12 17:08:41,2018-03-27,289.90,37.35,health_beauty,692.0,15.0,12.0,14.0,other,GO,curitiba,PR,1029.815339,15.0,29,14.0,2,6
110187,2018-07-25 16:18:46,2018-07-30 19:42:42,2018-08-13,89.90,18.73,furniture_decor,450.0,16.0,16.0,16.0,other,MG,other,SP,425.698898,5.0,18,13.0,7,2
110188,2017-07-24 11:29:43,2017-08-10 19:42:10,2017-08-23,29.40,17.92,auto,700.0,16.0,7.0,11.0,other,BA,sao paulo,SP,1140.413178,17.0,30,13.0,7,0
110190,2017-03-15 11:41:46,2017-03-20 10:34:54,2017-04-03,45.00,10.96,books_technical,300.0,28.0,2.0,18.0,santo andre,SP,other,SP,24.849316,5.0,19,14.0,3,2
110191,2018-08-10 21:14:35,2018-08-21 04:16:31,2018-08-30,44.99,22.25,sports_leisure,600.0,30.0,20.0,20.0,other,MG,other,SC,1395.064940,10.0,19,9.0,8,4
110192,2018-08-10 21:14:35,2018-08-21 04:16:31,2018-08-30,44.99,22.25,sports_leisure,600.0,30.0,20.0,20.0,other,MG,other,SC,1395.064940,10.0,19,9.0,8,4
110193,2018-07-01 10:23:10,2018-07-09 15:06:57,2018-07-20,79.00,14.13,construction_tools_lights,750.0,30.0,28.0,28.0,other,SP,other,SP,218.573444,8.0,19,11.0,7,6
110194,2018-07-24 09:46:27,2018-08-02 22:47:35,2018-08-16,399.00,45.07,furniture_decor,2100.0,80.0,8.0,30.0,other,CE,other,SP,2317.965145,10.0,23,13.0,7,1
110195,2018-05-22 21:13:21,2018-06-12 23:11:29,2018-06-08,219.90,24.12,furniture_decor,5900.0,41.0,21.0,41.0,other,MG,sao paulo,SP,826.147138,21.0,16,5.0,5,1
110196,2018-05-15 17:41:00,2018-05-21 14:31:41,2018-05-29,15.50,12.79,perfumery,83.0,17.0,8.0,13.0,sao bernardo do campo,SP,ribeirao preto,SP,288.967171,6.0,13,7.0,5,1


In [34]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer

#Here, we use a standard scaler for the numeric features and one-hot encoding for the categorical features.  
preprocess = make_column_transformer(
    (StandardScaler(),['price', 'freight_value', 'product_weight_g', 'product_length_cm','product_height_cm','product_width_cm','distance'], ),
    (OneHotEncoder(categories="auto",drop="first"),['customer_city','seller_city', 'month', 'product_category_name','day_of_week'], )
)

X = preprocess.fit_transform(delivered_items)
X.shape

(107004, 124)

In [35]:
delivered_items.shape

(107004, 20)

In [36]:
y = delivered_items.target

In [37]:
y.shape

(107004,)

In [38]:
from sklearn.model_selection import train_test_split
#Get x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

#And check the shape
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(85603, 124) (85603,)
(21401, 124) (21401,)


## Model 1

For our first model, we use a Stochastic Gradient Descent regressor. 

In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import SGDRegressor
model_1 = SGDRegressor(loss='squared_error', max_iter=1000, random_state=42)
model_1.fit(x_train,y_train)

SGDRegressor(random_state=42)

In [ ]:
def get_model_scores(model):
    predictions = model.predict(x_test)
    rmse_1 = mean_squared_error(y_test, predictions, squared=False)
    baseline_rmse = mean_squared_error(delivered_items['target'], delivered_items['olist_prediction'], squared=False)
    mae_1 = mean_absolute_error(y_test, predictions)
    baseline_mae = mean_absolute_error(delivered_items['target'], delivered_items['olist_prediction'])
    print("mae_{}: {}".format(model, mae_1))
    print("baseline_mae: {}".format(baseline_mae))
    print("rmse_{}: {}".format(model, rmse_1))
    print("baseline_rmse: {}".format(baseline_rmse))
    return [mae_1,rmse_1]

In [ ]:
get_model_scores(model_1)

mae_SGDRegressor(random_state=42): 5.1719010487196675
baseline_mae: 12.802829800755111
rmse_SGDRegressor(random_state=42): 8.209318726562527
baseline_rmse: 15.208898267794748


[5.1719010487196675, 8.209318726562527]

Here, we see that our initial model is already an improvement over the e-commerce website's predicted delivery time. The existing predictions have an average mean absolute error (MAE) of 12.8 days, meaning that the predictions are off by almost two weeks on average. Our first model has an MAE of 5.2, which means orders are off by around 5 days on average. 

In [ ]:
results_df = pd.DataFrame(np.zeros(shape=(7,2)))
results_df.index=[1,2,3,4,5,6,7]
results_df.columns = ["MAE", "RMSE"]
results_df.index.rename("Model",inplace=True)

In [ ]:
results_df.loc[1] = get_model_scores(model_1)

mae_SGDRegressor(random_state=42): 5.1719010487196675
baseline_mae: 12.802829800755111
rmse_SGDRegressor(random_state=42): 8.209318726562527
baseline_rmse: 15.208898267794748


In [ ]:
results_df

,MAE,RMSE
Model,,
1,5.171901,8.209319
2,0.000000,0.000000
3,0.000000,0.000000
4,0.000000,0.000000
5,0.000000,0.000000
6,0.000000,0.000000
7,0.000000,0.000000


## Model 2

For our second model, we will try to use a grid search to select more optimal parameters for the SGDRegressor. 

In [ ]:
from sklearn.model_selection import GridSearchCV

model_2_grid = SGDRegressor(random_state=42, loss='squared_error')

parameters = {
    'penalty': ['l2', 'l1', 'elasticnet'],
    'alpha': [.0001, .001, .01],
    'max_iter': [100,1000,10000]
    
}

model_2_grid = GridSearchCV(model_2_grid,parameters, cv = 3, n_jobs = -1)
                      

model_2_grid.fit(x_train, y_train)

print(model_2_grid.best_score_)
print(model_2_grid.best_params_)

/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/leo/anaconda3/lib/python3.9/si

0.24059339935289448
{'alpha': 0.0001, 'max_iter': 100, 'penalty': 'elasticnet'}


/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [ ]:
model_2 = SGDRegressor(loss='squared_error', max_iter=100, random_state=42, penalty='elasticnet', alpha=.0001)
model_2.fit(x_train,y_train)

/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:1549: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(max_iter=100, penalty='elasticnet', random_state=42)

In [ ]:
results_df.loc[2] = get_model_scores(model_2)

mae_SGDRegressor(max_iter=100, penalty='elasticnet', random_state=42): 5.167386607579634
baseline_mae: 12.802829800755111
rmse_SGDRegressor(max_iter=100, penalty='elasticnet', random_state=42): 8.205611615831009
baseline_rmse: 15.208898267794748


In [ ]:
results_df

,MAE,RMSE
Model,,
1,5.171901,8.209319
2,5.153059,8.179208
3,0.000000,0.000000
4,0.000000,0.000000
5,0.000000,0.000000
6,0.000000,0.000000
7,0.000000,0.000000


Above, we see that using the grid search resulted in marginally better results. 

## Model 3

For our next model, we will try using a random forest regressor. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_3 = RandomForestRegressor(random_state=42,n_estimators=30)
model_3.fit(x_train,y_train)


RandomForestRegressor(n_estimators=30, random_state=42)

In [ ]:
results_df.loc[3] = get_model_scores(model_3)

mae_RandomForestRegressor(n_estimators=30, random_state=42): 4.569093943185574
baseline_mae: 12.802829800755111
rmse_RandomForestRegressor(n_estimators=30, random_state=42): 7.5865402776468995
baseline_rmse: 15.208898267794748


Above, we see that the random forest model was a substantial improvement over the SGD models. The MAE is now 4.6 instead of 5.2. 

## Model 4

For the next model, we will use a grid search to improve the hyperparameter selection for the random forest. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

model_4_grid = RandomForestRegressor(random_state=42, criterion='squared_error')

parameters = {
    'n_estimators': [50,100, 150],
    'min_samples_split': [2,5,10],
    
}

model_4_grid = GridSearchCV(model_4_grid,parameters, cv = 3, n_jobs = -1)
                      

model_4_grid.fit(x_train, y_train)

print(model_4_grid.best_score_)
print(model_4_grid.best_params_)

0.3161606389810195
{'min_samples_split': 5, 'n_estimators': 150}


In [ ]:
model_4 = RandomForestRegressor(n_estimators=150, min_samples_split=5,random_state=42, criterion='squared_error')
model_4.fit(x_train,y_train)

RandomForestRegressor(min_samples_split=5, n_estimators=150, random_state=42)

In [ ]:
results_df.loc[4] = get_model_scores(model_4)

mae_RandomForestRegressor(min_samples_split=5, n_estimators=150, random_state=42): 4.526839317175524
baseline_mae: 12.802829800755111
rmse_RandomForestRegressor(min_samples_split=5, n_estimators=150, random_state=42): 7.473020815728326
baseline_rmse: 15.208898267794748


Above, we see that the grid search provided a small improvement in the MAE and RMSE scores. 

## Model 5

For our fifth model, we decided to try using [xgboost](https://xgboost.readthedocs.io/en/stable/), a popular and efficient machine learning algorithm using gradient boosting. 

In [ ]:
import xgboost as xg
model_5 = xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 100, seed = 42)
model_5.fit(x_train,y_train)

/home/leo/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [ ]:
results_df.loc[5] = get_model_scores(model_5)

mae_XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None): 4.7803286320075244
baseline_mae: 12.802829800755111
rmse_XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max

Above, we see that the xgboost model had an MAE of around 4.8, which is not quite as good as the random forests, but considerably better than the SGD models. However, xgboost was much faster to implement than the random forests. 

## Model 6

For the next model, we used a grid search to improve the hyperparameter selection of the xgboost model. 

In [ ]:
from sklearn.model_selection import GridSearchCV

xgb1 = xg.XGBRegressor()

parameters = {'objective':['reg:squarederror'],
              'learning_rate': [.1, .3, .5], #so called `eta` value
              'max_depth': [4, 6, 8],
              'min_child_weight': [1, 3],
              'subsample': [0.9, 1],
              'n_estimators': [10]}

xgb_grid = GridSearchCV(xgb1,parameters, cv = 3, n_jobs = -1)
                      

xgb_grid.fit(x_train, y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

/home/leo/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/leo/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/leo/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/leo/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the app

0.28611489856575784
{'learning_rate': 0.3, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 10, 'objective': 'reg:squarederror', 'subsample': 1}


In [ ]:
model_6 = xg.XGBRegressor(objective ='reg:squarederror', n_estimators = 100, seed = 42, max_depth=8, learning_rate=.3, min_child_weight=1, subsample=1)
model_6.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.3, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [ ]:
results_df.loc[6] = get_model_scores(model_6)

mae_XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.3, max_delta_step=0,
             max_depth=8, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None): 4.77300783017191
baseline_mae: 12.802829800755111
rmse_XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.3, max_delta_step=0,
             max

Here, we see that the grid search barely improved the results of the xgboost model.

## Model 7

For our final model, we tried using a neural network. This time, we started with a grid search to select hyperparameters. 

In [ ]:
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import GridSearchCV

parameters = {
     'solver':[('adam')],
     'activation': ('logistic', 'tanh', 'relu'),
    'alpha': (.001, .0001, .00001),
     'max_iter': (100, 300),
     'hidden_layer_sizes': [(100,50),(100,100),(200,100)]
}
gs_clf = GridSearchCV(MLPRegressor(random_state=5),parameters,cv=3,n_jobs=-1)
gs_clf.fit(x_train, np.ravel(y_train))

/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/leo/anaconda3/lib/python3.9/site-packages/skle

GridSearchCV(cv=3, estimator=MLPRegressor(random_state=5), n_jobs=-1,
             param_grid={'activation': ('logistic', 'tanh', 'relu'),
                         'alpha': (0.001, 0.0001, 1e-05),
                         'hidden_layer_sizes': [(100, 50), (100, 100),
                                                (200, 100)],
                         'max_iter': (100, 300), 'solver': ['adam']})

In [ ]:
print(gs_clf.best_params_)

{'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (100, 50), 'max_iter': 100, 'solver': 'adam'}


In [ ]:
model_7 = MLPRegressor(solver='adam', alpha=.001, hidden_layer_sizes=(100,50), max_iter=100, activation='logistic')
model_7.fit(x_train,y_train)

/home/leo/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:684: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(activation='logistic', alpha=0.001, hidden_layer_sizes=(100, 50),
             max_iter=100)

In [ ]:
results_df.loc[7] = get_model_scores(model_7)

mae_MLPRegressor(activation='logistic', alpha=0.001, hidden_layer_sizes=(100, 50),
             max_iter=100): 4.915359177062052
baseline_mae: 12.802829800755111
rmse_MLPRegressor(activation='logistic', alpha=0.001, hidden_layer_sizes=(100, 50),
             max_iter=100): 7.903881216640155
baseline_rmse: 15.208898267794748


Here, we see that the results for the neural network were better than the SGD models, but worse than the random forests and xgboost models. 

### Summary

In [ ]:
results_df.index = ['SGD','SGD-Grid','Random Forest','Random Forest-Grid','XG','XG-Grid','Neural Net']

In [57]:
results_df = pd.read_csv('./delivery_time_results.csv')

In [59]:
results_df.columns = ["Model","MAE","RMSE"]

In [61]:
results_df = results_df.set_index('Model')

In [65]:
present_df = results_df.iloc[[1,3,5,6]]

In [67]:
present_df.index = ['SGD','Random Forest','XGBoost','Neural Net']

In [68]:
present_df

,MAE,RMSE
SGD,5.167387,8.205612
Random Forest,4.526839,7.473021
XGBoost,4.773008,7.661160
Neural Net,4.915359,7.903881


In [54]:
results_df

,Unnamed: 0,MAE,RMSE
0,SGD,5.171901,8.209319
1,SGD-Grid,5.167387,8.205612
2,Random Forest,4.569094,7.586540
3,Random Forest-Grid,4.526839,7.473021
4,XG,4.780329,7.678326
5,XG-Grid,4.773008,7.661160
6,Neural Net,4.915359,7.903881


Overall, the best model was the Random Forest using a grid search to tune hyperparameters. This model had an MAE of approximately 4.5. This means that the predicted delivery date was off by around 4.5 days on average. While this is still a considerable amount, it is an improvement over the e-commerce company's current predictions, which have an MAE of 12.8.

There are several ways,however, that our model could be improved in the future. First, if we had the exact addresses for customers and sellers we would be able to calculate exact distances instead of an approximation based on city. Additionally, we could use a mapping website to generate the typical driving time between the two addresses instead of the distance. As roads do not generally go in a straight line between two points, the driving time would likely be a better measure than distance. We could also improve the model by penalizing early predictions more than late predictions. In a business setting, we want the model to be accurate, but we would also want the model to generally be more conservative. Customers are generally more unhappy if a delivery arrives late than if it arrives early, so it would be particularly important for the model to not predict a delivery time that is earlier than the estimated delivery time. 